In [18]:
from __future__ import print_function
import mxnet as mx
from mxnet import autograd, gluon, nd

In [19]:
data_ctx = mx.cpu()
model_ctx = mx.gpu()

In [20]:
num_inputs = 2
num_outputs = 1
num_samples = 10000

def real_fn(X):
    return 2 * X[:, 0] - 3.4 * X[:, 1] + 4.2

X = nd.random_normal(0, 1, shape=(num_samples, num_inputs))
noise = 0.01 * nd.random_normal(0, 1, shape=(num_samples, ))
y = real_fn(X) + noise

In [21]:
y


[5.6596646 3.6175725 4.730915  ... 1.0712998 7.9948    7.261293 ]
<NDArray 10000 @cpu(0)>

In [22]:
batch_size = 4
train_data = gluon.data.DataLoader(gluon.data.ArrayDataset(X, y), batch_size = batch_size, shuffle=True)

In [23]:
net = gluon.nn.Dense(units=1, in_units=2)

In [26]:
net.collect_params().initialize(mx.init.Normal(sigma=1.), ctx=model_ctx)

In [27]:
net.weight.data()


[[0.28419438 0.8668934 ]]
<NDArray 1x2 @gpu(0)>

In [28]:
loss = gluon.loss.L2Loss()

In [29]:
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.0001})